In [453]:
import pandas as pd
import numpy as np
import h5py
import matplotlib
import math

#small_data = '/Users/brandonchea/Desktop/Fall_2022/CS170/Project2/Datasets/CS170_Small_Data__91.txt'
small_text = "/Users/brandonchea/Desktop/Fall_2022/CS170/Project2/Datasets/test/Sue/CS170_Small_Data__96.txt"

small_data_df = pd.read_csv(small_text,
                            sep ='  ',
                            engine = 'python',
                            header = None)

large_text = '/Users/brandonchea/Desktop/Fall_2022/CS170/Project2/Datasets/CS170_Large_Data__103.txt'


large_data_df = pd.read_csv(large_text, 
                         sep='  ',
                         engine = 'python',
                         header=None)


#small_arr = small_data
small_data = small_data_df.to_numpy()

large_data = large_data_df.to_numpy()

#we take the dataframe, we are provided a set of features
#we wish to add a feature
#we do this by going through the data frame, add the feature_to_add to the current set
#then we remove the features we don't want
#def leave_one_out_cross_validation(data_df,current_set,feature_to_add):

def stub(data_df,current_set,feature_to_add):
    return np.random.rand()

def dist(a,b):
    diff = a[1:] - b[1:]
    return(np.dot(diff,diff))

#small_data_df = small_data_df.drop([2,4,5],axis=1)
#def nn(data_df,current_set,feature_to_add): #this is correct
def nn(data_df): #this is correct
    
    number_correctly_classified = 0
    
    for i in range(0,len(data_df)):
        #object_to_classify = data_df.iloc[i]
        object_to_classify = data_df[i,1:]
        #label_object_to_classify = data_df.iloc[i][0]
        label_object_to_classify = data_df[i,0]
        
        nearest_neighbor_distance = float('inf')
        nearest_neighbor_location = float('inf')
        
        for k in range(0,len(data_df)):
            if k!=i:            #object_to_classify,data_df.iloc[k]
                #distance = dist(object_to_classify,data_df[k,1:])
                distance = dist(data_df[i],data_df[k])
                if distance < nearest_neighbor_distance:
                    nearest_neighbor_distance = distance
                    nearest_neighbor_location = k 
                    #data_df.iloc[nearest_neighbor_location][0]
                    nearest_neighbor_label = data_df[nearest_neighbor_location,0]
                    #print(nearest_neighbor_label)               
        if label_object_to_classify == nearest_neighbor_label:
            number_correctly_classified = number_correctly_classified +1 
        
    accuracy = number_correctly_classified / len(data_df)
    #print(accuracy)
    return accuracy

#print(type(small_arr))


def leave_one_out_cross_validation(data_df,current_set,feature_to_add):
    current_set.append(feature_to_add)
    
    drops = []
    x = 1
    for x in range(1,data_df.shape[1]):
        if x not in current_set:
            drops.append(x)
            
        
    #data_df.loc[:,drops] = 0
    data_df[:,drops]=0

    accuracy = nn(data_df)
    return accuracy


current_set = [1,3]
feature_to_add = 6
print(leave_one_out_cross_validation(small_data,current_set,feature_to_add))


print(type(small_data))


0.94
<class 'numpy.ndarray'>


In [456]:
def feature_search_demo(data):
    current_set_of_features = []
    acc_at_level=[]
    for i in range(1,data.shape[1]):#level
        print("On the ",i," level of the search tree")
        feature_to_add_at_this_level = []
        best_so_far_acc = 0
        for k in range(1,data.shape[1]):#features
            print(current_set_of_features)
            if k not in current_set_of_features:
                print("Considering adding feature ",k)
                
                accuracy = leave_one_out_cross_validation(data,current_set_of_features,k)
                acc_at_level.append(accuracy)
                print("We have accuracies",acc_at_level)
                
                if (accuracy > best_so_far_acc):
                    best_so_far_acc = accuracy
                    feature_to_add_at_this_level = k
                    
        current_set_of_features.append(feature_to_add_at_this_level)
        print("On level ",i," I added feature",feature_to_add_at_this_level," to current set with accuracy",best_so_far_acc)
        print(current_set_of_features)
        
feature_search_demo(small_data)

On the  1  level of the search tree
[]
Considering adding feature  1
We have accuracies [0.874]
[1]
Considering adding feature  2
We have accuracies [0.874, 0.874]
[1, 2]
Considering adding feature  3
We have accuracies [0.874, 0.874, 0.874]
[1, 2, 3]
Considering adding feature  4
We have accuracies [0.874, 0.874, 0.874, 0.874]
[1, 2, 3, 4]
Considering adding feature  5
We have accuracies [0.874, 0.874, 0.874, 0.874, 0.874]
[1, 2, 3, 4, 5]
Considering adding feature  6
We have accuracies [0.874, 0.874, 0.874, 0.874, 0.874, 0.874]
On level  1  I added feature 1  to current set with accuracy 0.874
[1, 2, 3, 4, 5, 6, 1]
On the  2  level of the search tree
[1, 2, 3, 4, 5, 6, 1]
[1, 2, 3, 4, 5, 6, 1]
[1, 2, 3, 4, 5, 6, 1]
[1, 2, 3, 4, 5, 6, 1]
[1, 2, 3, 4, 5, 6, 1]
[1, 2, 3, 4, 5, 6, 1]
On level  2  I added feature []  to current set with accuracy 0
[1, 2, 3, 4, 5, 6, 1, []]
On the  3  level of the search tree
[1, 2, 3, 4, 5, 6, 1, []]
[1, 2, 3, 4, 5, 6, 1, []]
[1, 2, 3, 4, 5, 6, 1, []]
[1,

In [ ]:
def knn(data_df):
#def leave_one_out_cross_validation(data_df,current_set,feature_to_add):
    number_correctly_classified = 0
    
    for i in range(1,len(data_df)):
        object_to_classify = data_df.iloc[i]
        label_object_to_classify = data_df.iloc[i][0]
        
        nearest_neighbor_distance = float('inf')
        nearest_neighbor_location = float('inf')
        
        for k in range(1,len(data_df)):
            if k!=i:
                distance = math.sqrt(sum(object_to_classify - data_df.iloc[k])**2)
                if distance < nearest_neighbor_distance:
                    nearest_neighbor_distance = distance
                    nearest_neighbor_location = k
                    nearest_neighbor_label = data_df.iloc[nearest_neighbor_location][0]
                #print("Ask if ", i, " is neighest neighbor with",k)
        
        #print("Looping over i, at the ",i,"location")
        #print("The ",i,"th object is in class",label_object_to_classify)
        if label_object_to_classify == nearest_neighbor_label:
            number_correctly_classified +=1
        #print("Object ",i, " is class ", label_object_to_classify)
        #print("It's nearest neighbor is ", nearest_neighbor_location, " which is in class ", nearest_neighbor_label)
    accuracy = number_correctly_classified / len(data_df)
    #print(accuracy)
    return accuracy
        
knn(small_data_df)